In [ ]:
!pip install langchain-core langchain-openai \
langchain-community GitPython \
langchain-chroma tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.8 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT']   = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY']    = userdata.get('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT']    = 'default'

os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

In [ ]:
# LangChain의 GitLoader를 사용해 LangChain 공식 문서를 로드
from langchain_community.document_loaders import GitLoader

def file_filter(file_path: str) -> bool:
  return file_path.endswith(".mdx")

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

439


In [ ]:
# OpenAI의 Embedding 모델을 사용해 문서를 벡터화
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small", chunk_size=16)
db = Chroma.from_documents(documents, embeddings)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''
다음 문맥만을 고려해 질문에 답하세요.

문맥: """
{context}
"""

질문: {question}
''')

model = ChatOpenAI(model="gpt-5-nano", temperature=0)
retriever = db.as_retriever()

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | prompt | model | StrOutputParser()

chain.invoke("LangChain의 개요를 알려줘")

'다음은 주어진 문맥에 따른 LangChain의 개요입니다.\n\n- 정의\n  - LangChain은 대형 언어 모델(LLM)을 중심으로 애플리케이션을 개발할 수 있게 해 주는 프레임워크입니다.\n\n- 핵심 목적과 특징\n  - 표준 인터페이스 제공: LLM, 임베딩 모델, 벡터 스토어 등 다양한 공급자와의 표준화된 인터페이스를 제공합니다.\n  - LLM 애플리케이션 라이프사이클 관리: 개발(구성 요소와 흐름), 프로덕션화(모니터링·평가), 배포(생산 API/어시스턴트로의 전환)를 간소화합니다.\n  - 아키텍처·생태계 구성: 여러 오픈소스 라이브러리로 구성되며, 핵심은 langchain-core, integrations, langchain, langchain-community, langgraph입니다.\n  - 오케스트레이션 강화: LangGraph를 통해 상태를 갖는 에이전트, 스트리밍, 휴먼 인 더 루프를 포함한 복잡한 흐름을 구성할 수 있습니다. LangGraph는 LangChain과 함께 사용하거나 단독으로 사용할 수 있습니다.\n  - 관찰성 및 평가: LangSmith를 이용해 추적, 모니터링, 평가를 제공하고 observability를 강화합니다.\n  - 생산성 도구·플랫폼: LangSmith(관찰/평가), LangGraph Platform(프로덕션 배포)을 통해 신뢰성 있는 운영이 가능하게 합니다.\n\n- 구성 요소 및 아키텍처\n  - langchain-core: 챗모델 등 기본 추상화.\n  - Integration 패키지: langchain-openai, langchain-anthropic 등 다양한 공급자 연동 구현.\n  - langchain: 체인, 에이전트, 검색 전략 등 애플리케이션의 코그니티브 구조 구성 요소.\n  - langchain-community: 커뮤니티가 만든 추가 연동들.\n  - langgraph: 생산성 높은 애플리케이션을 위한 오케스트레이션 프레임워크(퍼시스턴스, 스트리밍, 메모리 등 포함).\n\n

In [ ]:
# Advanced RAG
# 1. 검색 쿼리 기법
# 1.1 HyDE(Hypothetical Document Embeddings)
# 단순한 RAG 구성에서는 사용자의 질문에 대해 임베딩 벡터의 유사도가 높은 문서를 검색함.
# 그러나 실제로 검색하고자 하는 것은 질문과 유사한 문서가 아니라 답변과 유사한 문서.
# 이를 위해 사용자의 질문에 대해 LLM에 가상의 답변을 추론하게 하고, 그 출력을 임베딩 벡터의 유사도 검색에 사용함.
hypothetical_prompt = ChatPromptTemplate.from_template("""
다음 질문에 한 문장으로 답하세요.

질문: {question}
""")

hypothetical_chain = hypothetical_prompt | model | StrOutputParser()

hyde_rag_chain = {
    "question": RunnablePassthrough(),
    "context": hypothetical_chain | retriever,
} | prompt | model | StrOutputParser()

hyde_rag_chain.invoke("LangChain의 개요를 알려줘")


'다음은 주어진 문맥을 바탕으로 한 LangChain의 개요입니다.\n\n- 목적\n  - 파이썬 패키지와 LangChain 생태계를 통해 개발자가 의사결정을 필요로 하는 AI 애플리케이션을 쉽게 구축하도록 돕는 것.\n\n- 핵심 아이디어\n  - 표준화된 컴포넌트 인터페이스: 다양한 모델과 컴포넌트의 API 차이를 줄이고 서로 바꿔 쓰기 쉽게 함.\n  - 오케스트레이션: 여러 컴포넌트와 모델을 연결해 복잡한 제어 흐름을 구성.\n  - 관찰성 및 평가: 애플리케이션의 동작을 관찰하고 평가하기 쉬운 도구를 제공.\n\n- 생태계 구성\n  - LangChain 패키지: 표준 인터페이스를 제공하는 핵심 구성요소 모음.\n  - LangGraph: 구성요소를 노드/엣지 형태로 연결해 복잡한 흐름을 제어하고, 지속성(persistence), 기억(memory), human-in-the-loop 등 기능과 함께 사용할 수 있음.\n  - LangSmith: 관찰성, 추적, 테스트, 평가를 지원하는 observability 플랫폼.\n\n- 주요 기능/예시\n  - 표준 인터페이스: 모든 챗 모델이 BaseChatModel 같은 공통 인터페이스를 구현하고, 도구 바인딩(tool calling)이나 구조화된 출력 등 공통 기능을 제공.\n  - 도구 바인딩 및 구조화된 출력: 모델에 도구를 연결하거나 출력 형식을 정의하는 일반적 방법 제공.\n  -Retriever 인터페이스: RAG 등에서 데이터 소스와의 연결을 표준화.\n\n- 사용 방식의 유연성\n  - 대부분의 구성요소를 서로 독립적으로도 사용할 수 있으며, 필요에 따라 조합해 복잡한 애플리케이션을 구축할 수 있음.\n\n- 장점 요약\n  - 공급자 간 스위칭이 쉬워짐(유연한 교체성).\n  - 스트리밍, 도구 호출 등 고급 기능의 일관된 접근 방식 제공.\n  - LangGraph를 통해 복잡한 흐름의 오케스트레이션 가능.\n  - LangSmith로 우수한 관찰성과 평가 체계 확보.\n\n간단히 말해, 

In [ ]:
# 복수 검색 쿼리 생성 방법
from pydantic import BaseModel, Field

class QueryGenerationOutput(BaseModel):
  queries: list[str] = Field(..., description="검색 목록 쿼리")

query_generation_prompt = ChatPromptTemplate.from_template("""\
질문에 대해 벡터 데이터베이스에서 관련 문서를 검색하기 위한
3개의 서로 다른 검색 쿼리를 생성하세요.
거리 기반 유사성 검색의 한계를 극복하기 위해
사용자의 질문에 대해 여러 관점을 제공하는 것이 목표입니다.

질문: {question}
""")

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    | (lambda x: x.queries)
)

# retriever.map()에서는 일반적으로 retriever가 str을 받아 list[Document]를 반환하는 것과 달리,
# list[str]을 받아 list[list[Document]]를 반환하도록 변환함.
multi_query_rag_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map(),
} | prompt | model | StrOutputParser()

multi_query_rag_chain.invoke("LangChain의 개요를 알려줘")

'LangChain은 대형 언어 모델(LLM) 기반 애플리케이션 개발을 위한 오픈소스 프레임워크입니다. 주요 목표는 LLM, 임베딩 모델, 벡터 저장소 등 다양한 컴포넌트를 표준 인터페이스로 제공해 개발 라이프사이클의 모든 단계를 간소화하는 것에 있습니다.\n\n주요 내용 요약\n- 개발(Development): LangChain의 오픈 소스 컴포넌트와 다양한 타사 연동을 활용해 애플리케이션을 구성합니다. LangGraph를 사용해 상태가 있는 에이전트를 만들고 스트리밍과 인간의 피드백 루프를 지원합니다.\n- 운영화/생산화(Productionization): LangSmith를 통해 애플리케이션의 추적, 모니터링, 평가를 수행하고 안정적으로 배포할 수 있도록 돕습니다.\n- 배포(Deployment): LangGraph Platform을 이용해 LangGraph 애플리케이션을 프로덕션용 API나 어시스턴트로 배포합니다.\n\n주요 구성\n- langchain-core: 챗 모델 및 기타 컴포넌트의 기본 추상화 제공.\n- Integration packages: 특정 공급자의 연동 패키지.\n- langchain: 애플리케이션의 고수준 구성요소 모음.\n- langchain-community: 커뮤니티가 유지하는 연동.\n- langgraph: 지속성, 스트리밍 등과 함께 작동하는 생산형 오케스트레이션 프레임워크.\n\n추가 정보\n- 표준 인터페이스: LangChain은 대형 언어 모델뿐만 아니라 임베딩 모델, 벡터 저장소 등과도 표준화된 방식으로 연동합니다.\n- 생태계/도구: LangSmith(추적 및 평가), LangGraph(그래프 기반 오케스트레이션) 등이 중심 역할을 합니다.\n- 언어 및 문서: Python 기반의 라이브러리로 주로 다뤄지며, API 참조, 튜토리얼, 가이드를 제공합니다.\n\n필요하면 위의 핵심 요점을 바탕으로 더 구체적인 부분(예: LangGraph의 기능이나 LangSmith의 평가 도구)에 대해 추가로 정리해 드리겠습니다.

In [ ]:
# 2. 검색 후 기법
# 2.1 RAG-Fusion
# 각 쿼리의 검색 결과를 프롬프트에 넣을 때는 어떤 순서로 정렬할 필요가 있음.
# 여러 검색 결과의 순서를 융합해 정렬하는 알고리즘으로 RFF(Reciprocal Rank Fusion)가 있음.
from langchain_core.documents import Document

def reciprocal_rank_fusion(
    retriever_outputs: list[list[Document]],
    k: int = 60,
) -> list[str]:
    # 각 문서의 콘텐츠(문자열)와 그 점수의 매핑을 저장하는 딕셔너리 준비
    content_score_mapping = {}

    # 검색 쿼리마다 반복
    for docs in retriever_outputs:
      # 검색 결과의 문서마다 반복
      for rank, doc in enumerate(docs):
        content = doc.page_content

        # 처음 등장한 콘텐츠인 경우 점수를 0으로 초기화
        if content not in content_score_mapping:
          content_score_mapping[content] = 0

        # (1 / (순위 + k)) 점수를 추가
        content_score_mapping[content] += 1 / (rank + k)

    # 점수가 큰 순서로 정렬
    ranked = sorted(content_score_mapping.items(), key=lambda x: x[1], reverse=True)
    return [content for content, _ in ranked]

rag_fusion_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map() | reciprocal_rank_fusion,
} | prompt | model | StrOutputParser()

rag_fusion_chain.invoke("LangChain의 개요를 알려줘")

'다음은 주어진 문맥에 기초한 LangChain의 개요입니다.\n\n- LangChain은 LLM(대형 언어모델) 기반 애플리케이션을 개발하기 위한 프레임워크로, 핵심 컴포넌트의 표준 인터페이스를 제공합니다.\n- 목적은 서로 다른 모델/서비스를 쉽게 교체하고, 다양한 구성요소를 조합해 복잡한 워크플로우를 구성하며, 개발 속도와 생산성을 높이는 것입니다.\n- 라이프사이클 관점에서의 핵심 아이디어:\n  - 개발(Development): LangChain의 오픈 소스 컴포넌트와 타사 연동으로 애플리케이션 작성 용이.\n  - 프로덕션화(Productionization): LangSmith를 통한 관찰성·평가 및 모니터링 기능 제공.\n  - 배포(Deployment): LangGraph Platform 등을 통한 생산환경 배포 지원.\n- 구성 요소 및 생태계:\n  - langchain-core: 채팅모델 등 기본 추상화 제공.\n  - Integrations: OpenAI, Anthropic 등 다양한 벤더의 연동 패키지.\n  - langchain: 체인, 에이전트, 검색 전략 등 애플리케이션의 코어 구성요소.\n  - langchain-community: 커뮤니티가 제공하는 추가 연동.\n  - langgraph: 복잡한 워크플로우를 그래프로 표현하는 오케스트레이션 프레임워크.\n  - LangSmith: 추적/평가/관찰성 도구로, 실행 과정의 로그와 메타데이터를 수집.\n- LCEL( LangChain Expression Language):\n  - 선언형(what) 방식으로 새로운 체인을 기존 Runnables로 구성하는 방식으로, 런타임 실행을 LangChain이 최적화하도록 돕습니다.\n- 비동기/병렬 실행, 스트리밍 지원 등 성능과 운영의 이점도 제공합니다.\n- 주로 파이썬 라이브러리 중심이며, 자바스크립트/JS 버전도 존재합니다(문맥상 파이썬에 초점을 둔 설명이 주를 이룸).\n\n필요하시면 위의 각 항목을 더 자세히 풀어 설명해 드리겠습니

In [ ]:
# 2.2 리랭크 모델
# 검색 결과를 다시 정렬하는 방법 중 하나로 리랭크 모델(머신러닝 모델)을 사용하는 방법.
# 임베딩 벡터의 유사도 검색보다 계산 비용이 높은 대신 랭킹 정확도가 높은 모델을 사용함.
# 따라서 계산 비용이 낮은 임베딩 벡터의 유사도 검색을 수행한 후 리랭크 모델을 적용함.
# Cohere 링랭크 모델은 현 시점에 무료 플랜으로 테스트 가능.

# 테스트는 미수행함.
!pip install langchain-cohere

os.environ['COHERE_API_KEY'] = userdata.get('COHERE_API_KEY')

from typing import Any
from langchain_cohere import CohereRerank
from langchain_core.documents import Document

def rerank(inp: dict[str, Any], top_n: int = 3) -> list[Document]:
  question = inp["question"]
  douments = inp["documents"]

  cohere_rerank = CohereRerank(model="rerank_multilingual-v3.0", top_n=top_n)

  return cohere_rerank.compress_documents(documents=documents, query=question)

  rerank_rag_chain = (
      {
          "question": RunnablePassthrough(),
          "documents": retriever,
      }
      | RunnablePassthrough.assign(context=rerank)
      | prompt | model | StrOutputParser()
  )

  rerank_rag_chain.invoke("LangChain의 개요를 알려줘")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.3/295.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 45.6 MB/s eta 0:00:00
  Attempting uninstall: httpx-sse
    Found existing installation: httpx-sse 0.4.1
    Uninstalling httpx-sse-0.4.1:
      Successfully uninstalled httpx-sse-0.4.1
ERROR: Operation cancelled by user


SecretNotFoundError: Secret COHERE_API_KEY does not exist.

In [ ]:
# 3. 복수 Retriever를 활용하는 기법
# 지금까지 설명한 예에서는 임베딩 벡터의 유사도 검색 Retriever만 사용함.
# 경우에 따라서는 여러 Retriever를 사용하는 것이 유용할 수 있음.
# 3.1 LLM에 의한 라우팅
from langchain_community.retrievers import TavilySearchAPIRetriever

langchain_document_retriever = retriever.with_config({"run_name": "langchain_document_retriever"})
web_retriever = TavilySearchAPIRetriever(k=3).with_config({"run_name": "web_retriever"})

from enum import Enum
from typing import Any

class Route(str, Enum):
  langchain_document = "langchain_document"
  web = "web"

class RouteOutput(BaseModel):
  route: Route

route_prompt = ChatPromptTemplate.from_template("""
질문에 답변하기 위해 적절한 Retriever를 선택하세요.

질문: {question}
""")

route_chain = (
    route_prompt
    | model.with_structured_output(RouteOutput)
    | (lambda x: x.route)
)

def routed_retriever(inp: dict[str, Any]) -> list[Document]:
  question = inp["question"]
  route = inp["route"]

  if route == Route.langchain_document:
    return langchain_document_retriever.invoke(question)
  elif route == Route.web:
    return web_retriever.invoke(question)

  raise ValueError(f"Unknown route: {route}")

route_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "route": route_chain
    }
    | RunnablePassthrough.assign(context=routed_retriever)
    | prompt | model | StrOutputParser()
)

route_rag_chain.invoke("LangChain의 개요를 알려줘")

'다음은 문맥에 근거한 LangChain의 개요입니다.\n\n- 정의: LangChain은 대형 언어 모델(LMM)을 기반으로 하는 애플리케이션을 개발하기 위한 프레임워크입니다.\n- 목적과 라이프사이클: 개발(구성 요소와 통합으로 애플리케이션 구축), 운영화(관찰과 평가로 최적화 및 안정적 배포), 배포(생산 환경용 API/Assistant으로 확장) 등 LLM 애플리케이션의 전체 수명주기를 단순화합니다.\n- 표준 인터페이스 및 확장성: LLM, 임베딩 모델, 벡터 스토어 등 여러 구성요소에 대한 표준 인터페이스를 제공하고, 수백 개의 프로바이더와의 통합을 지원합니다. 프로바이더 간 교체를 쉽게 할 수 있도록 설계되어 있습니다.\n- 아키텍처 구성요소: 오픈 소스 라이브러리로\n  - langchain-core: 챗 모델 및 기타 컴포넌트의 기본 추상화\n  - integration 패키지(langchain-openai 등): 주요 통합\n  - langchain: 체인, 에이전트, 검색 전략 등 애플리케이션의 인지 아키텍처 구성요소\n  - langchain-community: 커뮤니티가 유지하는 3자 통합\n  - langgraph: 생산 준비된 애플리케이션을 구성요소로 직렬화된 그래프로 오케스트레이션하는 프레임워크\n- 오케스트레이션과 생산성: LangGraph는 복잡한 흐름(다중 에이전트, 기억, 퍼시스턴스, 인간의 개입 등)을 관리하도록 설계되었으며, LinkedIn, Uber, Klarna, GitLab 등에서 생산형 에이전트에 사용됩니다. LangGraph Platform을 통해 프로덕션 API로도 활용 가능.\n- 관찰성과 평가: LangSmith를 통해 실행 흐름의 추적, 모니터링 및 평가를 지원합니다.\n- 생태계와 자료: LangChain은 LangSmith, LangGraph 등 생태계를 포함하며, Python 라이브러리 위주 문서가 주를 이룹니다(JS/타 언어 문서는 별도 사이트에서 제공).\n\n간단히 말해, LangChain은 L

In [ ]:
route_rag_chain.invoke("오늘 서울 날씨는?")

'오늘 서울의 시간별 날씨 예보는 다음과 같습니다:\n- 06:00 — 23°C, 바람 약 3 km/h\n- 09:00 — 26°C, 바람 약 3 km/h\n- 12:00 — 30°C, 바람 약 5 km/h\n- 15:00 — 29°C, 바람 약 12 km/h\n- 18:00 — 26°C, 바람 약 7 km/h\n\n대체로 오늘 낮 기온이 30°C 안팎까지 오르는 더운 날씨로 예상됩니다.'

In [ ]:
!pip install rank-bm25

In [ ]:
# 3.2 하이브리드 검색
# Embedding 모델로 생성하는 임베딩 벡터는 학습 데이터의 범위 내에서 의미가 유사한 텍스트의 경우
# 벡터 유사도가 높아지도록 설계되어 있음.
# Embedding 모델에서는 학습 데이터에 포함되지 않은 고유명사나 전문 용어의 유사도 검색은 어려움.
# 자연어 처리의 고전적인 기법으로 단어의 등장 빈도를 기반으로 텍스트의 유사도를 계산하는 방법이 있음.
# 유명한 예로 TF-IDF나 BM25(Elasticsearch의 기본 랭킹 알고리즘)가 있음.
# 즉, 고유명사나 전문 용어를 다를 때는 범용 Embedding 모델의 임베딩 벡터 유사도 검색보다
# TF-IDF나 BM25로 생성한 벡터의 유사도 검색이 더 효과적일 수 있음.
# TF-IDF나 BM25로 생성한 벡터는 많은 요소가 0이 되는 특성으로 인해 '희소 벡터'라고 불립니다.
# 반면, 임베딩 벡터는 '밀집 벡터'라고 불립니다.
from langchain_community.retrievers import BM25Retriever

chroma_retriever = retriever.with_config({"run_name": "chroma_retriever"})
bm25_retriever = BM25Retriever.from_documents(documents).with_config(
    {"run_name": "bm25_retriever"}
)

from langchain_core.runnables import RunnableParallel

hybrid_retriever = (
    RunnableParallel(
      {
          "chroma_documents": chroma_retriever,
          "bm25_documents": bm25_retriever,
      }
    )
    | (lambda x: [x["chroma_documents"], x["bm25_documents"]])
    | reciprocal_rank_fusion
)

hybrid_rag_chain = (
    {
        "question": RunnablePassthrough(),
        "context": hybrid_retriever,
    }
    | prompt | model | StrOutputParser()
)

hybrid_rag_chain.invoke("LangChain의 개요를 알려줘")

'다음은 주어진 문맥에 따른 LangChain의 개요입니다.\n\n- LangChain은 대형언어모델(LLM)로 구동되는 애플리케이션을 개발하기 위한 프레임워크입니다. LLM, 임베딩 모델, 벡터 스토어 등과의 표준 인터페이스를 제공하고 수백 개의 프로바이더와의 통합을 지원합니다.\n\n- 라이프사이클 전반을 아우르는 개발·운영 도구를 제공합니다.\n  - 개발(Development): LangChain의 오픈소스 컴포넌트와 통합을 사용해 애플리케이션을 구축하고, LangGraph를 통해 상태를 가진 에이전트를 스트리밍 및 휴먼 인 더 루프를 지원하며 구성합니다.\n  - 운영(Productionization): LangSmith를 이용해 애플리케이션을 점검·모니터링·평가하고, 지속적으로 개선 및 배포를 가능하게 합니다.\n  - 배포(Deployment): LangGraph Platform을 통해 LangGraph 애플리케이션을 프로덕션 API나 어시스턴트로 전환합니다.\n\n- 아키텍처 및 구성요소\n  - langchain-core: 챗 모델 등 기본 추상화 제공.\n  - Integration 패키지들(e.g., langchain-openai, langchain-anthropic 등): 주요 통합들을 경량 패키지로 제공.\n  - langchain: 체인, 에이전트, 검색 전략 등 애플리케이션의 인지 구조를 구성.\n  - langchain-community: 커뮤니티가 관리하는 3자 통합.\n  - langgraph: 생산환경용 조정 프레임워크로, 지속성, 스트리밍, 기타 핵심 기능을 포함한 다중 컴포넌트의 조합을 가능하게 함. LangGraph는 독립적으로도 사용할 수 있습니다.\n\n- 생태계와 지원 도구\n  - LangSmith: 모델 애플리케이션의 추적·평가 및 observability 기능 제공.\n  - LangGraph: 상태가 있는 멀티 에이전트 애플리케이션 구축을 돕는 orchestration 도구.\n  - 문서와 학습 자료: 개념